### KDD 

In [2]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank\\Test\\KDD'

In [3]:
cd ../..

C:\Users\gauthieca\Desktop\Code_ Thèse\bandits-to-rank


In [4]:
from __future__ import print_function
import sys 
import time 
from pyclick.click_models.Evaluation import LogLikelihood, Perplexity
from pyclick.click_models.UBM import UBM
from pyclick.click_models.DBN import DBN
from pyclick.click_models.SDBN import SDBN
from pyclick.click_models.DCM import DCM
from pyclick.click_models.CCM import CCM
from pyclick.click_models.CTR import DCTR, RCTR, GCTR
from pyclick.click_models.CM import CM
from pyclick.click_models.PBM import PBM
from pyclick.utils.Utils import Utils
from pyclick.utils.YandexRelPredChallengeParser import YandexRelPredChallengeParser

from pyclick.search_session.SearchResult import SearchResult
from pyclick.click_models.task_centric.TaskCentricSearchSession import TaskCentricSearchSession

from random import sample
from random import random

In [23]:
import pandas as pd

In [11]:
import sys
sys.path.insert(0, 'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank')

from bandits_to_rank.sampling.pbm_inference import *
import numpy as np

In [26]:
cd ..

C:\Users\gauthieca\Desktop\Code_ Thèse\bandits-to-rank


## Import de KDD light

In [27]:
KDD_train_light = pd.read_csv('KDD_filtered.csv')

In [31]:
KDD_train_light.shape

(3015547, 13)

### Fonction for get matrice

In [32]:
def get_list_clean_ad(df_query):

    ###get ad over 1000 :
    df_query_group_ad_Pos = df_query.groupby(["QueryID",'AdId','Position']).count()
    over_10000 =  df_query_group_ad_Pos['Impression']>=1000
    df_query_over1000 = df_query_group_ad_Pos[over_10000]

    ### get ad on all 3position : 
    df_query_over1000_ad = df_query_over1000.groupby(["QueryID",'AdId']).count()
    on_3pos =  df_query_over1000_ad['Impression']==3
    df_query_group_ad_3 = df_query_over1000_ad[on_3pos]
    
    ###List Add :
    ads_list = list(df_query_group_ad_3.index.get_level_values('AdId'))
    return ads_list 

In [66]:
def buil_mat (df_mat):
    nb_ad,nb_pos =df_mat.shape
    mat =np.zeros((nb_ad, nb_pos), dtype=np.int)
    dict_ad_ind={}
    ad_num=0
    for index, row in df_mat.iterrows():  
        for pos in range(nb_pos):
            dict_ad_ind[index]=ad_num
            mat[ad_num][pos]=row[pos]
        ad_num+=1
    return dict_ad_ind,mat

In [68]:
def getmat (query,df_full ):
    ###filter by query
    df_query= df_full[df_full.QueryID==query]
    ### filter by ad
    ad_list = get_list_clean_ad(df_query)
    df_query_clean = df_query[df_query.AdId.isin(ad_list)]
    
    #### GET DF impression and click
    df_mat_query_impr = df_query_clean[['AdId','Position','Impression']].groupby(['AdId','Position'],as_index = False).count().pivot('AdId','Position').fillna(0)
    df_mat_query_clic = df_query_clean[['AdId','Position','Click']].groupby(['AdId','Position'],as_index = False).sum().pivot('AdId','Position').fillna(0)
    
    
    #### Transform to np_array
    dict_ad_ind,mat_query_impr = buil_mat(df_mat_query_impr)
    dict_ad_ind,mat_query_clic = buil_mat(df_mat_query_clic)
    return mat_query_impr,mat_query_clic,dict_ad_ind
    

In [72]:
impr_19,click_19,_ = getmat(19,KDD_train_light)

In [76]:
def get_param_query(query,df_full):
    impr,click,_ = getmat(query,df_full)
    nb_products,nb_positions = impr.shape
    svd =SVD(nb_products, nb_positions)
    svd.nb_views = impr
    svd.nb_clicks =click
    svd.learn()
   
    return  svd.get_params() 

In [86]:
query = [19, 2, 10, 9, 7, 8, 4, 1]

In [87]:
dict_theta_query ={}
for i in query:
    theta, kappa =get_param_query(i,KDD_train_light)
    dict_theta_query[i]={'thetas':list(theta),'kappas':list(kappa)}

In [91]:
dict_theta_query

{19: {'thetas': [0.04998115200718376,
   0.03430311957880712,
   0.036602165932245315,
   0.04669622534961218,
   0.030629107242356162],
  'kappas': [1.0, 0.48574928517746124, 0.3297979789061402]},
 2: {'thetas': [0.01827178757558634,
   0.02353305657790025,
   0.018121746359756177,
   0.07734856183748726,
   0.016282720520416426],
  'kappas': [1.0, 0.5032179142509637, 0.4033327487828045]},
 10: {'thetas': [0.14847462536579176,
   0.004398044587847975,
   0.039165307602481596,
   0.11481401719638429,
   0.0739868806137541,
   0.013119969501046307],
  'kappas': [1.0, 0.41114941297751506, 0.27510543487320777]},
 9: {'thetas': [0.04024562589619809,
   0.06870495493362541,
   0.03836948985880075,
   0.02284041243892426,
   0.06149888422426908,
   0.017378239524063456],
  'kappas': [1.0, 0.5458576614906008, 0.529046315216675]},
 7: {'thetas': [0.04643163870552662,
   0.025341272333348094,
   0.026069485537007244,
   0.04198094853055038,
   0.024656101028562114,
   0.06674437580594327],
  'k

### Record param

In [98]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank\\Test\\KDD'

In [93]:
cd Test/KDD/

C:\Users\gauthieca\Desktop\Code_ Thèse\bandits-to-rank\Test\KDD


In [99]:
import json
with open('param_KDD.txt', 'w') as file:
    json.dump(dict_theta_query, file)

#### olllllllllddd

In [4]:
import zipfile
         
zip = zipfile.ZipFile('track2.zip')
zip.extract('track2/training.txt')

zip.close()

In [5]:
pwd

'C:\\Users\\gauthieca\\Desktop\\Code_ Thèse\\bandits-to-rank'

In [6]:
cd track2

C:\Users\gauthieca\Desktop\Code_ Thèse\bandits-to-rank\track2


In [8]:
import pandas as pd
df_train_KDD= pd.read_csv("training.txt", sep="\t",  names=['Click','Impression','DisplayURL','AdId','AdvertiserID','Depth','Position','QueryID','KeywordID','TitleID','DescriptionID','UserID '],dtype={'Click':int,'Impression':int,'Depth':int,'Position':int},nrows=100000000)  

MemoryError: 

In [ ]:
for line in open("training.txt"):
    print (line)

In [ ]:
def readInChunks(fileObj, chunkSize=1024):
    while True:
        data = fileObj.read(chunkSize)
        if not data:
            break
        while data[-1:] != '\n':
            data+=fileObj.read(1)
        yield data

In [8]:
df_train_KDD.shape

(5255063, 12)

In [10]:
df_train_KDD.head()

,Click,Impression,DisplayURL,AdId,AdvertiserID,Depth,Position,QueryID,KeywordID,TitleID,DescriptionID,UserID
0,0,1,4298118681424644510,7686695,385,3,3,1601,5521,7709,576,490234
1,0,1,4860571499428580850,21560664,37484,2,2,2255103,317,48989,44771,490234
2,0,1,9704320783495875564,21748480,36759,3,3,4532751,60721,685038,29681,490234
3,0,1,13677630321509009335,3517124,23778,3,1,1601,2155,1207,1422,490234
4,0,1,3284760244799604489,20758093,34535,1,1,4532751,77819,266618,222223,490234


In [96]:
df_train_KDD.groupby(["AdId","QueryID",'Depth','Position']).count().sort_values(by = 'Impression')

NameError: name 'df_train_KDD' is not defined